In [1]:
%load_ext autoreload
%autoreload 2

from transformers.generation import LogitsProcessor,LogitsProcessorList
from transformers import AutoModelForCausalLM, AutoTokenizer
import numpy as np
from scipy.stats import gumbel_l
from arsenal.maths.rvs import TruncatedDistribution
import copy
import torch
import tqdm
import scipy
from scipy.stats import gumbel_l, gumbel_r
from arsenal.maths.rvs import TruncatedDistribution
import transformers
from evaluate import load
import pickle
from collections import Counter, defaultdict
import ot
import sk2torch
from sklearn.linear_model import SGDClassifier
# import nn
import torch.nn as nn
import rlace

In [2]:
model_name="openai-community/gpt2-large"
model_name = "meta-llama/Meta-Llama-3-8B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name,trust_remote_code=True,torch_dtype=torch.float16,
                                             load_in_8bit=True,
                                             device_map='auto').eval()

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [3]:
@torch.no_grad()
def encode(model, tokenizer, text, batch_size,layer=-1, pooling="last"):
  encodings = []

  with torch.no_grad():
   for i in tqdm.tqdm(range(0, len(text), batch_size)):
    batch = text[i:i+batch_size]
    padded_tokens = tokenizer(batch, padding=True, return_tensors="pt", max_length=128, truncation=True).to("cuda")
    outputs = model(**padded_tokens, output_hidden_states=True)
    lengths = padded_tokens["attention_mask"].sum(axis=1).detach().cpu().numpy()

    hiddens = outputs.hidden_states[layer]
    hiddens = hiddens.detach()
    for h,l in zip(hiddens, lengths):
      if pooling == "last":
        h = h[l-1]
      elif pooling == "cls":
        h = h[0]
      elif pooling == "mean":
        h = h[:l].mean(axis=0)
      encodings.append(h.detach().cpu().numpy())

  return np.array(encodings)

In [4]:
with open("bios_data/bios_data/bios_train.pickle", "rb") as f:
    data = pickle.load(f)
    y = np.array([d["p"] for d in data])
    z = np.array([1 if d["g"] == "m" else 0 for d in data])
    texts = [d["text"] for d in data]

y_to_keep = ["professor", "physician", "attorney"]
idx_to_keep = [i for i in range(len(y)) if y[i] in y_to_keep]
y = y[idx_to_keep]
z = z[idx_to_keep]
texts = [texts[i] for i in idx_to_keep]

num_m, num_f = sum(z), len(z) - sum(z)
n = 10000#min(num_m, num_f)
idx_m = [i for i in range(len(z)) if z[i] == 1]
idx_f = [i for i in range(len(z)) if z[i] == 0]
idx = idx_m[:n] + idx_f[:n]
y = y[idx]
z = z[idx]
texts = [texts[i] for i in idx]


In [5]:
np.mean(z)

0.5

In [6]:
layer = 16
encodings = encode(model, tokenizer, texts, 32, layer=layer, pooling="mean")


  0%|          | 0/625 [00:00<?, ?it/s]We detected that you are passing `past_key_values` as a tuple and this is deprecated and will be removed in v4.43. Please use an appropriate `Cache` class (https://huggingface.co/docs/transformers/v4.41.3/en/internal/generation_utils#transformers.Cache)


100%|██████████| 625/625 [12:54<00:00,  1.24s/it]


In [7]:
# save the texts, encodings, and labels

with open(f"bios_data/bios_data/encodings_{layer}.pickle", "wb") as f:

    pickle.dump({"texts": texts, "encodings": encodings, "y": y, "z": z}, f)

In [73]:
# # import pca
# from sklearn.decomposition import PCA
# pca = PCA(n_components=64)
# encodings_pca = pca.fit_transform(encodings)

# output_rlace = rlace.solve_adv_game(encodings_pca, z, encodings_pca, z, rank=7, device="cpu", out_iters=15000, in_iters_adv=1, in_iters_clf=1, epsilon=0.0015, batch_size=256, evalaute_every=1000, optimizer_params_P={"lr": 0.005, "weight_decay": 1e-4}, optimizer_params_predictor={"lr": 0.005, "weight_decay": 1e-4})


3000/15000. Acc post-projection: 50.150%; best so-far: 50.150%; Maj: 50.000%; Gap: 0.150%; best loss: 0.7001; current loss: 0.7001:  20%|##        | 3000/15000 [00:24<01:37, 122.80it/s] 


In [79]:
# P = np.eye(64) - output_rlace["P"]
# # perform truncated svd and collect eigenvalues. use numpy.linalg.eigh

# output = np.linalg.eigh(P)
# eigenvalues = list(sorted(output[0]))[::-1]
# # plot eigenvalues
# import matplotlib.pyplot as plt

# plt.rcParams["font.family"] = "serif"
# import matplotlib.pyplot as plt
# plt.rcParams.update({'font.size': 15})

# plt.plot(eigenvalues[:16])
# plt.xlabel("$K$", fontsize=15)
# plt.ylabel("Eigenvalue", fontsize=15)
# # make xlabel ticks ot be intetgers from 1 to 16
# plt.xticks(np.arange(0, 16, 1))
# plt.savefig("eigenvalues.pdf", dpi=400)
# plt.show()


In [107]:
import scipy

def get_optimal_gaussian_transport_func(source_x, target_x):
      cov_source = np.cov(source_x.T).real + 1e-8
      cov_target = np.cov(target_x.T).real + 1e-8

      # optimal transport

      cov_source_sqrt = matrix_squared_root(cov_source)
      cov_source_sqrt_inv = matrix_inv_squared_root(cov_source) #scipy.linalg.inv(cov_source_sqrt)

      A = cov_source_sqrt_inv @ matrix_squared_root(cov_source_sqrt @ cov_target @ cov_source_sqrt) @ cov_source_sqrt_inv
      return A

def matrix_squared_root(A):
    return scipy.linalg.sqrtm(A)


def matrix_inv_squared_root(A):

    return np.linalg.inv(matrix_squared_root(A))

In [108]:
x_train_source = encodings[z==0,:][:]
x_train_target = encodings[z==1,:][:]
A = get_optimal_gaussian_transport_func(x_train_source, x_train_target)


In [109]:
mean_source = x_train_source.mean(axis=0)
mean_target = x_train_target.mean(axis=0)
encodings_transformed = encodings.copy()
encodings_transformed[z == 0] = mean_target + (encodings_transformed[z == 0] - mean_source) @ A
clf = SGDClassifier(loss="log_loss", max_iter=10000, tol=1e-4)
clf.fit(encodings, z)

SGDClassifier(loss='log_loss', max_iter=10000, tol=0.0001)

In [112]:
class InterventionModule(nn.Module):
    def __init__(self, mean_0, mean_1, A, mlp, alpha=1.0):
        super().__init__()
        self.mean_0 = torch.tensor(mean_0)
        self.mean_1 = torch.tensor(mean_1)
        self.A = torch.tensor(A)
        self.mlp = sk2torch.wrap(mlp)
        self.alpha = alpha
        # set requires_grad=False to all params of the mlp
        for p in self.mlp.parameters():
            p.requires_grad = False
    
    def to_cuda(self, device):
      self.A = self.A.to(device)
      self.mean_0 = self.mean_0.to(device)
      self.mean_1 = self.mean_1.to(device)
      self.mlp = self.mlp.to(device)

    def to_cpu(self):
      self.A = self.A.to("cpu")
      self.mean_0 = self.mean_0.to("cpu")
      self.mean_1 = self.mean_1.to("cpu")
      self.mlp = self.mlp.to("cpu")

    def forward(self, hidden_states):
        self.to_cuda(hidden_states.device)
        # fi hidden state is half, convert laso the params to half precision
        if hidden_states.dtype == torch.float16:
          self.A = self.A.half()
          self.mean_0 = self.mean_0.half()
          self.mean_1 = self.mean_1.half()
          self.mlp = self.mlp.half()

        
        preds = self.mlp(hidden_states)
        preds = preds[0]

        x = hidden_states.clone().reshape(-1, hidden_states.shape[-1])
        
        if self.alpha != 0:
            x[preds == 0] = self.alpha*self.mean_1 + (x[preds == 0] - self.alpha*self.mean_0)@self.A

        #print("Steering {} samples".format((preds == 0).sum()))
        x = x.reshape(hidden_states.shape)
        return x

def insert_intervention(model, model_name, intervention, layer, after_layer_norm=False, replace_existing=False):
    if "gpt2" in model_name.lower():
        # if the mlp is already a Sequential object, do nothing
        if isinstance(model.transformer.h[layer].mlp, torch.nn.Sequential):
            return
        if not replace_existing:
            model.transformer.h[layer].mlp = torch.nn.Sequential(model.transformer.h[layer].mlp, intervention)
        else:
            
            model.transformer.h[layer].mlp = torch.nn.Sequential(model.transformer.h[layer].mlp[:-1], intervention)

    elif "llama" in model_name.lower():
        # if the mlp is already a Sequential object, do nothing
        if isinstance(model.model.layers[layer].post_attention_layernorm, torch.nn.Sequential):
            return
        if not replace_existing:
            model.model.layers[layer].post_attention_layernorm = torch.nn.Sequential(model.model.layers[layer].post_attention_layernorm, intervention, model.model.layers[layer].post_attention_layernorm)
        else:
            model.model.layers[layer].post_attention_layernorm = torch.nn.Sequential(model.model.layers[layer].post_attention_layernorm[0], intervention, model.model.layers[layer].post_attention_layernorm[0])

    else:
        raise NotImplementedError("Only GPT2 is supported")


def remove_intervention(model, model_name, layer):
    if "gpt2" in model_name.lower():
        # if the mlp is not a Sequential object, do nothing
        if not isinstance(model.transformer.h[layer].mlp, torch.nn.Sequential):
            return
        model.transformer.h[layer].mlp = model.transformer.h[layer].mlp[0]

    elif "llama" in model_name.lower():
        # if the mlp is not a Sequential object, do nothing
        if not isinstance(model.model.layers[layer].post_attention_layernorm, torch.nn.Sequential):
            return
        model.model.layers[layer].post_attention_layernorm = model.model.layers[layer].post_attention_layernorm[0]

    else:
        raise NotImplementedError("Only GPT2 is supported")

In [179]:
remove_intervention(model, model_name, layer)


In [163]:
intervention_module = InterventionModule(mean_source, mean_target, A, clf, alpha=0.0)
remove_intervention(model, model_name, layer)
insert_intervention(model, model_name, intervention_module, layer, replace_existing=False,after_layer_norm=False)

In [120]:
from transformers import pipeline
generator = pipeline('text-generation', model=model, tokenizer=tokenizer)
# generate text
prompt = "Janet Kohn is a professor at NYU"
generator(prompt, max_length=75, num_return_sequences=1, do_sample=False)

[{'generated_text': 'Janet Kohn is a professor at NYU’s Steinhardt School of Culture, Education, and Human Development. She is a former Fulbright scholar and has been a visiting professor at the University of Paris. She has published numerous articles and books on the history of education and women’s education. She is the author of The Education of the Negro Prior to 1861 ('}]

In [119]:
text = "Janet Kohn is a professor at NYU's Stern School of Business. She lives in New York and has four children."
vocab_size = len(tokenizer.get_vocab())

In [180]:
from sampling import counterfactual_generation
counterfactual_text = counterfactual_generation(model, tokenizer, "A new day has come", vocab_size,prompt="A new")

  8%|▊         | 10683/128256 [00:02<00:23, 5032.40it/s]


KeyboardInterrupt: 

In [178]:
counterfactual_text

['A new component named as Bottle \xa0']

In [155]:
model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear8bitLt(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear8bitLt(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear8bitLt(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear8bitLt(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear8bitLt(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear8bitLt(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear8bitLt(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )


In [189]:
class GumbelProcessor(LogitsProcessor):
    def __init__(self, precomputed_noise=None,noise=0, replaced_pairs=None):
        self.precomputed_noise = precomputed_noise
        self.i=0
        self.replaced_pairs=replaced_pairs
        # set np random seed

        
        #np.random.seed(noise)
        self.noises = []

    def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor):
        self.i += 1
        if self.precomputed_noise is not None:
            return scores + self.precomputed_noise[self.i - 1]
        
        gumbel = np.random.gumbel(loc=0.0, scale=1.0, size=scores.shape)
        self.noises.append(gumbel)
        return scores + gumbel


def sample_from_truncated_gumbel(cdf_a,b,gumbel):
    cdf_b = gumbel.cdf(b)
    u = np.random.uniform(0, 1)
    return gumbel.ppf(cdf_a + u * (cdf_b - cdf_a))

def sample_gumbel(model, tokenizer, gumbel_processor, prompt):

    inputs = tokenizer(prompt, return_tensors="pt")
    generate_ids = model.generate(inputs.input_ids, max_length=64, logits_processor=[gumbel_processor],
                                   do_sample=False, num_return_sequences=1, pad_token_id=tokenizer.eos_token_id,)
    return tokenizer.batch_decode(generate_ids, skip_special_tokens=True)


def sample_from_truncated_gumbel_vectorized(cdf_a, b_array, gumbel):
    cdf_b = gumbel.cdf(b_array)  # Compute CDF for an array of b values
    u = np.random.uniform(0, 1, size=b_array.shape)  # Generate uniform random values of the same shape as b_array
    return gumbel.ppf(cdf_a + u * (cdf_b - cdf_a))  # Apply the inverse CDF to each element in the array

def counterfactual_generation(model, tokenizer, sentence, vocab_size, prompt=None):
    tokens = tokenizer(sentence, return_tensors="pt").input_ids
    logits = model(tokens).logits.detach().cpu().numpy()
    all_gumbel_noise = []

    cdf_a = gumbel_r.cdf(-500.0)

    for i, w in enumerate(tokens[0][1:]):
        # Generate Gumbel noise for each token in the vocabulary
        gumbel_noises = np.random.gumbel(size=vocab_size)

        # Calculate the logits differences
        logit_diffs = logits[0][i] - logits[0][i][w]

        # Apply vectorized truncated Gumbel sampling to the entire vocabulary
        truncated_gumbels = sample_from_truncated_gumbel_vectorized(cdf_a, gumbel_noises + logit_diffs, gumbel_r)

        # Ensure the original token keeps its specific noise
        truncated_gumbels[w.detach().cpu().numpy().item()] = gumbel_noises[w.detach().cpu().numpy().item()]
        
        all_gumbel_noise.append(truncated_gumbels)

    # Add a bias to the EOS token to make it more likely at the end
    eos = tokenizer.eos_token_id
    noise = np.zeros(vocab_size)
    noise[eos] = 500.0
    all_gumbel_noise.append(noise)

    all_gumbel_noise = np.array(all_gumbel_noise)

    # Convert Gumbel noise to a tensor
    processor = GumbelProcessor(precomputed_noise=torch.tensor(all_gumbel_noise))

    first_word = sentence.split(" ")[0]
    prompt = first_word if prompt is None else prompt
    return sample_gumbel(model, tokenizer, processor, prompt)



In [191]:
counterfactual_generation(model, tokenizer, "A new day has come", vocab_size, prompt="A new day has")

(128256,)
(128256,)
(128256,)
(128256,)
(128256,)


['A new day has \xa0 dawn for the']

In [168]:
model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear8bitLt(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear8bitLt(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear8bitLt(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear8bitLt(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear8bitLt(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear8bitLt(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear8bitLt(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )


In [169]:
all_gumbel_noise = [np.random.gumbel(size=vocab_size) for _ in range(64)]
processor = GumbelProcessor(precomputed_noise=torch.tensor(all_gumbel_noise))
vocab_size=128256
prompt = "Janet Kohn is a professor at"
print(sample_gumbel(model, tokenizer, processor, prompt))
processor = GumbelProcessor(precomputed_noise=torch.tensor(all_gumbel_noise))
print(sample_gumbel(model, tokenizer, processor, prompt))
#print(sample_gumbel(model, tokenizer, processor, prompt))

/home/shauli/miniconda3/envs/py39/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:540: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/shauli/miniconda3/envs/py39/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:545: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/home/shauli/miniconda3/envs/py39/lib/python3.9/site-packages/transformers/generation/utils.py:1797: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sur

['Janet Kohn is a professor at Kent State University. Her research focuses on information systems in higher education, modeling educational systems, and online communities. Dr. Kohn earned her doctorate degree in computer and information sciences at the University of Alabama.\nRichard Andersen is an Associate Professor in the School of Computer Science and']
['Janet Kohn is a professor at Kent State Law School. She is a graduate of DePaul College of Law. She also teaches at Capital Law School. She is a member of the bars of Ohio and Georgia. She co-authored "Civil Procedure for the Paralegal" with Daryl Waugh. The']
